In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
spam = pd.read_csv('spam.csv')
spam.head()

In [ ]:
spam.shape

In [ ]:
# Codificando a 'Category'
labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(spam['Category'])
print(y)

In [ ]:
mensagens = spam['Message'].values

X_train, X_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)

In [ ]:
# Tokenização das mensagens
token = Tokenizer(num_words=1000)
token.fit_on_texts(X_train)
token.fit_on_texts(X_test)
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [ ]:
# Vai mostrar uma lista de listas, em que cada uma das listas internas é uma sentença, em que cada número dela é uma palavra
print(X_train)

In [ ]:
# Precisamos deixar todas as listas com o mesmo tamanho
X_train = pad_sequences(X_train, padding="post", maxlen=500)
X_test = pad_sequences(X_test, padding="post", maxlen=500)

In [ ]:
# Verificando o tamnaho do vocabulário
len(token.word_index)

In [ ]:
X_train

In [ ]:
# Criando o modelo de RNA e aplicando o Word Embedding
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index),output_dim=50,input_length=500))
modelo.add(Flatten())
modelo.add(Dense(units=10, activation="relu"))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1, activation='sigmoid'))

In [ ]:
modelo.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
modelo.summary()

In [ ]:
modelo.fit(X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Acurácia: ", accuracy)

In [ ]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

In [ ]:
prev = (nova_previsao > 0.5)
print(prev)

In [ ]:
# Não vai gerar uma CM válida porque o modelo não foi criado de forma correta, sendo possível ver na tabela de Summary, que mostra que não há parâmetros nas camadas do modelo
cm = confusion_matrix(y_test, prev)
print(cm)